In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [140]:
# capacity of stations
C = [10,10,10,10];

# demand of stations
d = [[[0,10,0,5] [0,0,0,0] [0,5,0,0] [20,0,0,0]];;;
    [[0,0,0,0] [0,0,0,0] [30,0,0,0] [0,10,0,0]];;;
    [[0,0,0,5] [0,0,15,0] [0,0,0,0] [0,0,0,0]]];

# proximity of stations
X = [[0,1,1,1] [1,0,1,1] [1,1,0,1] [1,1,1,0]];

# capacity of stations
y1 = [6,2,4,3];

# number of vans
K = 1;

# capacity of vans
S = 20;

# distance of stations
D = [[0,0.5,1,0.5] [2,0,0.5,1] [0.5,1,0,0.5] [1,2,1,0]];

# number of stations
n_stations = 4;


In [135]:
d[:,:,2]

4×4 Matrix{Int64}:
 0  0  30   0
 0  0   0  10
 0  0   0   0
 0  0   0   0

In [162]:
function first_model(C,D,X,y1,K,S,d,n_stations)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 100)
    M=30
    lambda=0 # to be tuned
    n_hours = 3
    # Decision variables
    @variable(model, x[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Bin)
    @variable(model, 0 <= z[1:n_stations, 1:n_stations, 1:K, 1:n_hours], Int)
    @variable(model, 0 <= y[1:n_stations, 1:n_hours]) # Int by definition of the constraints
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:n_hours], Int) # Removed Int
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:n_hours])

    # Add constraints
    @constraint(model, [i in 1:n_stations, t in 1:n_hours], y[i,t] <= C[i])
    @constraint(model, [i in 1:n_stations, k in 1:K, t in 1:n_hours], x[i,i,k,t] == 0)
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)-sum(z[i,j,k,t] for j in 1:n_stations, k in 1:K)+sum(z[j,i,k,t] for j in 1:n_stations, k in 1:K))
    @constraint(model, [i in n_stations+1:n_stations, t in 2:n_hours], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations], y[i,1] == y1[i])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:n_hours], z[i,j,k,t] <= S)
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 2:n_hours], sum(w[j,i,t] for j in 1:n_stations) - sum(w[i,j,t] for j in 1:n_stations)<= C[i] - y[i,t])
    @constraint(model, [k in 1:K, t in 1:n_hours], sum(x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations) <= 1)
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:n_hours], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:n_hours], x[i,j,k,t] <= X[i,j])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 2:n_hours], u[i,j,t] >= d[i,j,t] - w[i,j,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, t in 2:n_hours], u[i,j,t] >= w[i,j,t] - d[i,j,t])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:n_hours)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:n_hours))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), value.(w), value.(u), value.(y), value.(z), objective_value(model)
end

first_model (generic function with 1 method)

In [163]:
x,w,u,y,z,obj=first_model(C,D,X,y1,K,S,d,n_stations)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-15
Set parameter TimeLimit to value 100
Objective value: 38.0


([0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;;; 0.0 -0.0 -0.0 -0.0; 0.0 0.0 -0.0 -0.0; -0.0 1.0 0.0 0.0; 0.0 0.0 -0.0 0.0;;;; 0.0 -0.0 0.0 -0.0; 0.0 0.0 1.0 -0.0; -0.0 -0.0 0.0 -0.0; -0.0 -0.0 -0.0 0.0], [-0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0;;; -0.0 -0.0 10.0 -0.0; -0.0 -0.0 -0.0 8.0; -0.0 -0.0 -0.0 -0.0; 7.0 -0.0 -0.0 -0.0;;; -0.0 -0.0 1.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 10.0 -0.0 -0.0; 2.0 -0.0 -0.0 -0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0;;; -0.0 0.0 20.0 0.0; 0.0 -0.0 0.0 2.0; 0.0 0.0 -0.0 0.0; 7.0 0.0 0.0 -0.0;;; -0.0 0.0 1.0 0.0; 0.0 -0.0 0.0 0.0; 0.0 5.0 -0.0 0.0; 3.0 0.0 0.0 -0.0], [6.0 2.9999999999999996 4.0; 2.0 8.0 0.0; 4.0 0.0 9.0; 3.0 4.0 2.0], [-0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0;;;; -0.0 0.0 -0.0 -0.0; -0.0 -0.0 -0.0 -0.0; -0.0 14.0 -0.0 -0.0; 0.0 -0.0 0.0 -0.0;;;; -0.0 0.0 0.0 0.0; -0.0 -0.0 18.0 0.0; -0.0 -0.0 -0.0 -0.0

In [166]:
sum(y[:,3])

15.0

In [167]:
function summary(x,w,u,y,z,obj)
    n,_,K,n_hours = size(x)
    for h=1:n_hours
        println("Hour ", h)
        for i=1:n
            println("Station ", i, " has ", y[i,h], " bikes")
        end
    end
end
summary(x,w,u,y,z,obj)



Hour 1
Station 1 has 6.0 bikes
Station 2 has 2.0 bikes
Station 3 has 4.0 bikes
Station 4 has 3.0 bikes
Hour 2
Station 1 has 2.9999999999999996 bikes
Station 2 has 8.0 bikes
Station 3 has 0.0 bikes
Station 4 has 4.0 bikes
Hour 3
Station 1 has 4.0 bikes
Station 2 has 0.0 bikes
Station 3 has 9.0 bikes
Station 4 has 2.0 bikes


In [161]:
z[:,:,1,2]

4×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 5.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [119]:
y[1,3]

8.0